### CONFIG

In [67]:
import pdfplumber
import re
import json
import os
import PyPDF2
import re
import pandas as pd
import sys
import os


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [68]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [69]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

### METHODS

In [70]:
# ESTRAGGO LO SCHEMA in modo da aiutare LLM a generare le domande

def get_database_schema(driver) -> str:
    """Estrae automaticamente lo schema del database, incluse le proprietà dei nodi e delle relazioni."""
    with driver.session() as session:
        # Query per ottenere i tipi di nodi
        node_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS labels
        RETURN COLLECT(labels) AS types
        """
        
        # Query per ottenere i tipi di relazioni e la loro struttura
        rel_query = """
        MATCH (a)-[r]->(b)
        WITH DISTINCT labels(a) AS sourceLabels, type(r) AS relType, labels(b) AS targetLabels
        RETURN sourceLabels, relType, targetLabels
        """
        
        # Query per ottenere le proprietà dei nodi
        node_properties_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS labels, keys(n) AS properties
        UNWIND properties AS property
        RETURN labels, property
        """
        
        # Query per ottenere le proprietà delle relazioni
        rel_properties_query = """
        MATCH ()-[r]->()
        WITH DISTINCT type(r) AS relType, keys(r) AS properties
        UNWIND properties AS property
        RETURN relType, property
        """
        
        # Esegui le query separatamente
        node_types = session.run(node_query).single()['types']
        relationships = list(session.run(rel_query))
        
        # Ottieni le proprietà dei nodi
        node_properties = list(session.run(node_properties_query))
        
        # Ottieni le proprietà delle relazioni
        rel_properties = list(session.run(rel_properties_query))
        
        # Costruisci una descrizione dettagliata dello schema
        schema_description = "Schema del database:\n\nNodi:\n"
        for node_type in node_types:
            schema_description += f"- {':'.join(node_type)}\n"
            # Aggiungi le proprietà dei nodi
            for node_property in node_properties:
                if set(node_type).intersection(set(node_property['labels'])):
                    schema_description += f"    Proprietà: {node_property['property']}\n"
        
        schema_description += "\nRelazioni e loro struttura:\n"
        for rel in relationships:
            source = ':'.join(rel['sourceLabels'])
            target = ':'.join(rel['targetLabels'])
            schema_description += f"- ({source})-[:{rel['relType']}]->({target})\n"
            # Aggiungi le proprietà delle relazioni
            for rel_property in rel_properties:
                if rel_property['relType'] == rel['relType']:
                    schema_description += f"    Proprietà: {rel_property['property']}\n"
            
        return schema_description

In [71]:
schema = get_database_schema(driver)
print(schema)

# Write the schema to a text file
with open("schema_as_is.txt", "w", encoding="utf-8") as file:
    file.write(schema)

Schema del database:

Nodi:
- Tecnica
    Proprietà: nome
    Proprietà: nome
    Proprietà: caratteristica
    Proprietà: nome
    Proprietà: tipo
    Proprietà: nome
    Proprietà: tipo
    Proprietà: caratteristica
    Proprietà: nome
    Proprietà: uso
    Proprietà: nome
    Proprietà: uso
    Proprietà: applicazione
    Proprietà: nome
    Proprietà: uso
    Proprietà: effetto
    Proprietà: nome
    Proprietà: Svantaggi
    Proprietà: nome
    Proprietà: descrizione
    Proprietà: nome
    Proprietà: tipo
    Proprietà: Svantaggi
    Proprietà: nome
    Proprietà: uso
    Proprietà: effetto
    Proprietà: Svantaggi
    Proprietà: applicazione
    Proprietà: nome
    Proprietà: descrizione
    Proprietà: tipo
    Proprietà: Svantaggi
    Proprietà: nome
    Proprietà: uso
    Proprietà: effetto
    Proprietà: Svantaggi
    Proprietà: nome
    Proprietà: tipo
    Proprietà: uso
    Proprietà: caratteristica
    Proprietà: Svantaggi
    Proprietà: nome
    Proprietà: uso
    Propri

In [72]:
def run_query(query, params=None):
    """Esegue una query su Neo4j e restituisce i risultati"""
    with driver.session() as session:
        result = session.run(query, params or {})
        return [record for record in result]

def get_all_node_labels():
    """Ottiene tutte le etichette di nodi presenti nel database"""
    query = "CALL db.labels()"
    return [record["label"] for record in run_query(query)]

def get_all_relationship_types():
    """Ottiene tutti i tipi di relazioni presenti nel database"""
    query = "CALL db.relationshipTypes()"
    return [record["relationshipType"] for record in run_query(query)]

def get_node_properties(label):
    """Ottiene tutte le proprietà per un tipo di nodo"""
    query = """
    MATCH (n:{label}) 
    UNWIND keys(n) AS prop 
    RETURN DISTINCT prop
    """.format(label=label)
    return [record["prop"] for record in run_query(query)]

def get_relationship_properties(rel_type):
    """Ottiene tutte le proprietà per un tipo di relazione"""
    query = """
    MATCH ()-[r:{rel_type}]->() 
    UNWIND keys(r) AS prop 
    RETURN DISTINCT prop
    """.format(rel_type=rel_type)
    return [record["prop"] for record in run_query(query)]

def normalize_property_names(entity_type, properties, is_node=True):
    """Usa LLM per normalizzare i nomi delle proprietà"""
    entity_name = "nodo" if is_node else "relazione"
    
    prompt = f"""
    Normalizza i seguenti nomi di proprietà per {entity_name} di tipo '{entity_type}' in Neo4j.
    Proprietà attuali: {', '.join(properties)}
    
    Regole:
    1. Uniforme lettere maiuscole/minuscole (prima lettera minuscola)
    2. Unisci proprietà con significato simile (es. 'nome' e 'Nome')
    3. Formato: '<proprietà originale>|<proprietà normalizzata>'
    4. Per ogni proprietà originale, fornisci una sola proprietà normalizzata
    
    Rispondi con un elenco di mappature, una per riga.
    """
    
    # Utilizzo di LangChain AzureChatOpenAI - corretto per ottenere il contenuto
    response = llm.invoke(prompt)
    # Estrai il contenuto dal messaggio
    response_text = response.content if hasattr(response, 'content') else str(response)
    
    mappings = {}
    for line in response_text.strip().split('\n'):
        if '|' in line:
            original, normalized = line.split('|', 1)
            mappings[original.strip()] = normalized.strip()
    
    return mappings

def identify_entity_references(entity_type, properties, target_schema, is_node=True):
    """Identifica proprietà che dovrebbero essere relazioni ad altre entità"""
    entity_name = "nodo" if is_node else "relazione"
    
    # Prepara il target schema in formato leggibile
    schema_text = "Schema target:\n"
    for node_type, props in target_schema.get("nodes", {}).items():
        schema_text += f"- {node_type}: {', '.join(props)}\n"
    
    for rel_type, info in target_schema.get("relationships", {}).items():
        schema_text += f"- {info['source']}-[:{rel_type}]->{info['target']}: {', '.join(info.get('properties', []))}\n"
    
    prompt = f"""
    Analizza le proprietà del {entity_name} '{entity_type}' e identifica quali dovrebbero essere spostate come relazioni ad altre entità.
    
    Proprietà attuali: {', '.join(properties)}
    
    {schema_text}
    
    Per ogni proprietà che rappresenta un'entità esterna, indica:
    1. Nome proprietà originale
    2. Tipo di entità a cui dovrebbe essere collegata
    3. Tipo di relazione da creare
    4. Quale proprietà usare per identificare l'entità di destinazione
    
    Formato: '<proprietà>|<tipo entità target>|<tipo relazione>|<proprietà identificativa>'
    
    Rispondi con un elenco di mappature, una per riga. Se una proprietà non rappresenta un'entità esterna, non includerla.
    """
    
    # Utilizzo di LangChain AzureChatOpenAI - corretto per ottenere il contenuto
    response = llm.invoke(prompt)
    # Estrai il contenuto dal messaggio
    response_text = response.content if hasattr(response, 'content') else str(response)
    
    mappings = []
    for line in response_text.strip().split('\n'):
        parts = line.split('|')
        if len(parts) == 4:
            mappings.append({
                "property": parts[0].strip(),
                "target_entity": parts[1].strip(),
                "relationship_type": parts[2].strip(),
                "target_property": parts[3].strip()
            })
    
    return mappings

def parse_schema_file(file_path):
    """Analizza un file di schema e restituisce un dizionario strutturato"""
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    schema = {"nodes": {}, "relationships": {}}
    
    # Uso LLM per analizzare il file di schema
    prompt = f"""
    Analizza il seguente schema di database Neo4j e convertilo in un formato strutturato JSON.
    Per ogni tipo di nodo, elenca tutte le proprietà.
    Per ogni tipo di relazione, specifica i nodi di origine e destinazione e le eventuali proprietà.
    
    Schema:
    {content}
    
    Formato di output desiderato (esempio):
    {{
      "nodes": {{
        "Person": ["name", "age"],
        "Movie": ["title", "year"]
      }},
      "relationships": {{
        "ACTED_IN": {{
          "source": "Person",
          "target": "Movie",
          "properties": ["role"]
        }}
      }}
    }}
    
    Rispondi solo con il JSON strutturato.
    """
    
    # Utilizzo di LangChain AzureChatOpenAI - corretto per ottenere il contenuto
    response = llm.invoke(prompt)
    # Estrai il contenuto dal messaggio
    response_text = response.content if hasattr(response, 'content') else str(response)
    
    # Estrai il JSON dalla risposta
    try:
        # Cerca il JSON nella risposta
        json_text = response_text.strip()
        
        # Prova a trovare l'inizio e la fine del JSON
        start_idx = json_text.find('{')
        end_idx = json_text.rfind('}') + 1
        
        if start_idx >= 0 and end_idx > start_idx:
            json_str = json_text[start_idx:end_idx]
            result = json.loads(json_str)
            return result
        else:
            print("Non è stato possibile trovare JSON valido nella risposta")
            print(f"Risposta: {json_text}")
            return schema
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing dello schema: {e}")
        print(f"Risposta ricevuta: {response_text}")
        return schema

def migrate_node_properties(label, property_mappings):
    """Aggiorna i nomi delle proprietà dei nodi"""
    for old_prop, new_prop in property_mappings.items():
        if old_prop != new_prop:
            query = f"""
            MATCH (n:{label})
            WHERE n.{old_prop} IS NOT NULL
            SET n.{new_prop} = n.{old_prop}
            REMOVE n.{old_prop}
            """
            run_query(query)
            print(f"Migrata proprietà {old_prop} -> {new_prop} per nodi {label}")

def migrate_relationship_properties(rel_type, property_mappings):
    """Aggiorna i nomi delle proprietà delle relazioni"""
    for old_prop, new_prop in property_mappings.items():
        if old_prop != new_prop:
            query = f"""
            MATCH ()-[r:{rel_type}]->()
            WHERE r.{old_prop} IS NOT NULL
            SET r.{new_prop} = r.{old_prop}
            REMOVE r.{old_prop}
            """
            run_query(query)
            print(f"Migrata proprietà {old_prop} -> {new_prop} per relazioni {rel_type}")

def create_entity_relationships(label, entity_mappings):
    """Crea nuove entità e relazioni basate sulle proprietà"""
    for mapping in entity_mappings:
        prop = mapping["property"]
        target_entity = mapping["target_entity"]
        rel_type = mapping["relationship_type"]
        target_prop = mapping["target_property"]
        
        # Query per creare i nodi target e le relazioni
        query = f"""
        MATCH (source:{label})
        WHERE source.{prop} IS NOT NULL
        MERGE (target:{target_entity} {{{target_prop}: source.{prop}}})
        MERGE (source)-[:{rel_type}]->(target)
        REMOVE source.{prop}
        """
        run_query(query)
        print(f"Creata relazione {label}-[:{rel_type}]->{target_entity} da proprietà {prop}")


def process_schema_manually(schema_content):
    """Analizza manualmente il file di schema in caso di fallimento dell'LLM"""
    schema = {"nodes": {}, "relationships": {}}
    
    lines = schema_content.strip().split('\n')
    current_node = None
    current_relationship = None
    
    for line in lines:
        line = line.strip()
        
        # Controlla se è l'inizio della definizione di un nodo
        if line.startswith('- ') and ' Proprietà:' not in line:
            current_node = line[2:].strip()
            schema["nodes"][current_node] = []
            
        # Controlla se è una proprietà di un nodo
        elif line.startswith('Proprietà:') and current_node:
            prop = line[10:].strip()
            if prop and prop not in schema["nodes"][current_node]:
                schema["nodes"][current_node].append(prop)
            
        # Controlla se è l'inizio della definizione di una relazione
        elif line.startswith('- (') and ')-[' in line and ']->(' in line:
            rel_parts = line.replace('- ', '').split(')-[')
            source_node = rel_parts[0].replace('(', '').strip()
            
            rel_target_parts = rel_parts[1].split(']->(')
            rel_type = rel_target_parts[0].replace(':', '').strip()
            target_node = rel_target_parts[1].replace(')', '').strip()
            
            schema["relationships"][rel_type] = {
                "source": source_node,
                "target": target_node,
                "properties": []
            }
            current_relationship = rel_type
            
        # Controlla se è una proprietà di una relazione
        elif line.startswith('Proprietà:') and current_relationship:
            prop = line[10:].strip()
            if prop and prop not in schema["relationships"][current_relationship]["properties"]:
                schema["relationships"][current_relationship]["properties"].append(prop)
    
    return schema

In [73]:
target_schema = parse_schema_file("schema_to_be.txt")
print(target_schema)

{'nodes': {'Tecnica': ['nome', 'tipo', 'caratteristica', 'uso', 'applicazione', 'effetto', 'descrizione', 'Svantaggi', 'Vantaggi', 'comeFunziona'], 'Ristorante': ['nome', 'Chef', 'Location', 'description', 'cucina', 'voto'], 'Piatto': ['nome', 'numero'], 'Ingrediente': ['nome', 'tipo', 'origine', 'sapore', 'Aspetto', 'Proprietà'], 'Licenza': ['nome', 'Livello', 'Descrizione'], 'Pianeta': ['nome', 'ambiente', 'clima', 'intolleranzaLattosio', 'intolleranza', 'descrizione', 'caratteristiche', 'biomi', 'magia']}, 'relationships': {'OFFERTA': {'source': 'Ristorante', 'target': 'Piatto'}, 'CONTIENE': {'source': 'Piatto', 'target': 'Ingrediente'}, 'PREPARATO': {'source': 'Piatto', 'target': 'Tecnica'}, 'SITUATO': {'source': 'Ristorante', 'target': 'Pianeta', 'properties': ['descrizione']}, 'HA': {'source': 'Ristorante', 'target': 'Licenza', 'properties': ['Livello']}, 'DISTANZA': {'source': 'Pianeta', 'target': 'Pianeta', 'properties': ['km']}}}


In [21]:
from pprint import pprint
pprint(target_schema)

{'nodes': {'Ingrediente': ['nome',
                           'tipo',
                           'origine',
                           'sapore',
                           'Aspetto',
                           'Proprietà'],
           'Licenza': ['nome', 'Livello', 'Descrizione'],
           'Pianeta': ['nome',
                       'ambiente',
                       'clima',
                       'intolleranzaLattosio',
                       'intolleranza',
                       'descrizione',
                       'caratteristiche',
                       'ambiente',
                       'biomi',
                       'magia'],
           'Piatto': ['nome', 'numero'],
           'Ristorante': ['nome',
                          'Chef',
                          'Location',
                          'description',
                          'cucina',
                          'voto'],
           'Tecnica': ['nome',
                       'tipo',
                       'caratteri

In [74]:
# Get all node labels and relationship types
node_labels = get_all_node_labels()
print(node_labels)

['Tecnica', 'Licenza', 'Pianeta', 'Piatto', 'Ristorante', 'Ingrediente', 'Content', 'Tipo', 'Grado', 'Supervisione', 'Ambiente', 'Legge', 'Competenza', 'Organo', 'Principio', 'Stato', 'Concessione', 'Consultazione', 'Ente']


In [75]:
relationship_types = get_all_relationship_types()
print(relationship_types)

['DISTANZA', 'PREPARATO', 'OFFERTA', 'CONTIENE', 'SITUATO', 'HA', 'HAS_TIPO', 'HAS_GRADO', 'HAS_SUPERVISIONE', 'HAS_AMBIENTE', 'HAS_LEGGE', 'HAS_COMPETENZA', 'HAS_ORGANO', 'HAS_PRINCIPIO', 'HAS_STATO', 'HAS_CONCESSIONE', 'HAS_CONSULTAZIONE', 'HAS_ENTE', 'HA_TECHNICA', 'PROVENIENTE_DA']


In [76]:
# Process nodes
for label in node_labels:
    # Get current properties
    properties = get_node_properties(label)
    
    # Normalize property names
    property_mappings = normalize_property_names(label, properties)
    
    # Migrate properties
    migrate_node_properties(label, property_mappings)
    
    # Identify and create entity relationships
    entity_mappings = identify_entity_references(label, properties, target_schema)
    create_entity_relationships(label, entity_mappings)

Migrata proprietà Svantaggi -> svantaggi per nodi Tecnica
Migrata proprietà Vantaggi -> vantaggi per nodi Tecnica
Creata relazione Tecnica-[:HAS_TIPO]->TipoTecnica da proprietà tipo
Creata relazione Tecnica-[:HAS_CARATTERISTICA]->CaratteristicaTecnica da proprietà caratteristica
Creata relazione Tecnica-[:HAS_USO]->UsoTecnica da proprietà uso
Creata relazione Tecnica-[:HAS_APPLICAZIONE]->ApplicazioneTecnica da proprietà applicazione
Creata relazione Tecnica-[:HAS_EFFETTO]->EffettoTecnica da proprietà effetto
Creata relazione Tecnica-[:HAS_VANTAGGIO]->Vantaggio da proprietà Vantaggi
Creata relazione Tecnica-[:HAS_SVANTAGGIO]->Svantaggio da proprietà Svantaggi
Creata relazione Tecnica-[:HAS_FUNZIONAMENTO]->FunzionamentoTecnica da proprietà comeFunziona
Migrata proprietà Livello -> livello per nodi Licenza
Migrata proprietà Descrizione -> descrizione per nodi Licenza


KeyboardInterrupt: 

In [77]:
def find_closest_property(old_prop, target_properties):
    """Trova la proprietà più simile in target_properties usando Levenshtein distance in Cypher."""
    
    if not target_properties:
        return old_prop  # Se non ci sono proprietà target, restituisce quella originale

    query = f"""
    WITH {target_properties} AS targetProps
    UNWIND targetProps AS new_prop
    RETURN new_prop, apoc.text.distance('{old_prop}', new_prop) AS distance
    ORDER BY distance ASC
    LIMIT 1
    """
    result = run_query(query)
    return result[0]["new_prop"] if result else old_prop  # Se non trova nulla, ritorna il nome originale


In [78]:
def migrate_node_properties(label, schema_to_be):
    """Aggiorna i nomi delle proprietà dei nodi, mappandoli a quelli previsti nello schema_to_be."""
    target_properties = schema_to_be.get("nodes", {}).get(label, [])
    
    if not target_properties:
        print(f"⚠️ Nessuna proprietà target trovata per il nodo {label}, salto la migrazione.")
        return

    current_properties = get_node_properties(label)
    property_mappings = {}

    for old_prop in current_properties:
        if old_prop not in target_properties:
            # Trova la proprietà più simile
            new_prop = find_closest_property(old_prop, target_properties)
            if new_prop != old_prop:
                property_mappings[old_prop] = new_prop

    # Aggiorna le proprietà nel database
    for old_prop, new_prop in property_mappings.items():
        query = f"""
        MATCH (n:{label})
        WHERE n.{old_prop} IS NOT NULL
        SET n.{new_prop} = n.{old_prop}
        REMOVE n.{old_prop}
        """
        run_query(query)
        print(f"✅ Migrata proprietà {old_prop} -> {new_prop} per nodi {label}")


In [79]:
def migrate_properties_to_relationships(label, schema_to_be):
    """Converte proprietà in relazioni SOLO se previste in schema_to_be."""
    
    entity_mappings = identify_entity_references(label, get_node_properties(label), schema_to_be)

    for mapping in entity_mappings:
        prop = mapping["property"]
        target_entity = mapping["target_entity"]
        rel_type = mapping["relationship_type"]
        target_prop = mapping["target_property"]

        # Controlla se la relazione è prevista nello schema_to_be
        if rel_type not in schema_to_be.get("relationships", {}):
            print(f"⚠️ Relazione {label}-[:{rel_type}]->{target_entity} non prevista in schema_to_be, salto la trasformazione per {prop}.")
            continue  # Saltiamo questa migrazione

        query = f"""
        MATCH (source:{label})
        WHERE source.{prop} IS NOT NULL
        MATCH (target:{target_entity} {{{target_prop}: source.{prop}}})
        MERGE (source)-[:{rel_type}]->(target)
        REMOVE source.{prop}
        """
        run_query(query)
        print(f"✅ Creata relazione {label}-[:{rel_type}]->{target_entity} da proprietà {prop}")


In [80]:
def migrate_relationship_properties(rel_type, schema_to_be):
    """Aggiorna i nomi delle proprietà delle relazioni, mappandoli a quelli previsti nello schema_to_be."""
    target_properties = schema_to_be.get("relationships", {}).get(rel_type, {}).get("properties", [])
    
    if not target_properties:
        print(f"⚠️ Nessuna proprietà target trovata per la relazione {rel_type}, salto la migrazione.")
        return

    current_properties = get_relationship_properties(rel_type)
    property_mappings = {}

    for old_prop in current_properties:
        if old_prop not in target_properties:
            # Trova la proprietà più simile
            new_prop = find_closest_property(old_prop, target_properties)
            if new_prop != old_prop:
                property_mappings[old_prop] = new_prop

    # Aggiorna le proprietà nel database
    for old_prop, new_prop in property_mappings.items():
        query = f"""
        MATCH ()-[r:{rel_type}]->()
        WHERE r.{old_prop} IS NOT NULL
        SET r.{new_prop} = r.{old_prop}
        REMOVE r.{old_prop}
        """
        run_query(query)
        print(f"✅ Migrata proprietà {old_prop} -> {new_prop} per relazioni {rel_type}")


In [81]:
def delete_unmapped_entities(schema_to_be):
    """Elimina nodi e relazioni che non esistono nello schema target."""
    
    all_labels = get_all_node_labels()
    all_rel_types = get_all_relationship_types()
    
    # Elimina nodi non presenti nello schema target
    for label in all_labels:
        if label not in schema_to_be.get("nodes", {}):
            query = f"MATCH (n:{label}) DETACH DELETE n"
            run_query(query)
            print(f"Eliminati nodi di tipo {label} perché non presenti nello schema target")

    # Elimina relazioni non presenti nello schema target
    for rel_type in all_rel_types:
        if rel_type not in schema_to_be.get("relationships", {}):
            query = f"MATCH ()-[r:{rel_type}]->() DELETE r"
            run_query(query)
            print(f"Eliminate relazioni di tipo {rel_type} perché non presenti nello schema target")


In [82]:
def migrate_graph(schema_to_be):
    """Esegue la migrazione completa del grafo secondo lo schema target."""
    
    # 1. Normalizza le proprietà dei nodi
    for label in get_all_node_labels():
        migrate_node_properties(label, schema_to_be)

    # 2. Trasforma proprietà in relazioni
    for label in get_all_node_labels():
        migrate_properties_to_relationships(label, schema_to_be)

    # 3. Normalizza le proprietà delle relazioni
    for rel_type in get_all_relationship_types():
        migrate_relationship_properties(rel_type, schema_to_be)

    # 4. Elimina nodi e relazioni non presenti in schema_to_be
    delete_unmapped_entities(schema_to_be)

    print("✅ Migrazione completata!")


In [83]:
migrate_graph(target_schema)

✅ Migrata proprietà svantaggi -> Svantaggi per nodi Tecnica
✅ Migrata proprietà vantaggi -> Svantaggi per nodi Tecnica
✅ Migrata proprietà livello -> Livello per nodi Licenza
✅ Migrata proprietà descrizione -> Descrizione per nodi Licenza
⚠️ Nessuna proprietà target trovata per il nodo Content, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Tipo, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Grado, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Supervisione, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Ambiente, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Legge, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Competenza, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Organo, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Principio, salto la migrazione.
⚠️ Nessuna proprietà target trovata per il nodo Stato, s

In [ ]:

# Estrai le tecniche e crea nuovi nodi
with driver.session() as session:
    # 1. Recupera tutte le tecniche
    result = session.run("MATCH (t:Licenza) RETURN id(t) AS id, t.nome AS nome")
    
    # Set per memorizzare le tecniche uniche
    tecniche_uniche = set()
    ids_to_delete = []
    
    # 2. Estrai le tecniche dalle liste
    for record in result:
        node_id = record["id"]
        nome_tecniche = record["nome"]
        
        # Controlla se nome_tecniche è una lista
        if isinstance(nome_tecniche, list):
            ids_to_delete.append(node_id)
            for tecnica in nome_tecniche:
                tecniche_uniche.add(tecnica)
    
    # 3. Crea nodi per le tecniche uniche
    for tecnica in tecniche_uniche:
        session.run("CREATE (:Licenza {nome: $nome})", nome=tecnica)
    
    print(f"Create {len(tecniche_uniche)} tecniche uniche")
    
    # 4. Elimina i nodi originali con liste (opzionale)
    if ids_to_delete:
        for node_id in ids_to_delete:
            session.run("MATCH (t:Tecnica) WHERE id(t) = $id DETACH DELETE t", id=node_id)
        print(f"Eliminati {len(ids_to_delete)} nodi originali con liste")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 26, offset: 25} for query: 'MATCH (t:Licenza) RETURN id(t) AS id, t.nome AS nome'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 25, offset: 24} for query: 'MATCH (t:Tecnica) WHERE id(t) = $id DETACH DELETE t'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in 

Create 25 tecniche uniche
Eliminati 8 nodi originali con liste
